 ## Problem description

PetFinder.my uses a basic Cuteness Meter to rank pet photos. It analyzes picture composition and other factors compared to the performance of thousands of pet profiles. 

While this basic tool is helpful, it's still in an experimental stage and the algorithm could be improved. The participants needs to build an AI model using provided data to help make the tool better.  

**Task** 

The task is to predict engagement with a pet's profile( **Pawpularity** ) based on the photograph for that profile. 

**Data** 

The dataset for this competition comprises both images and tabular data(hand-labelled metadata for each photo). 

The train set contains 9912 pet photos 

The test set contains 8 pet photos
> NOTE: The actual test data comprises about **6800** pet photos similar to the training set photos. 


####  **Previous Notebooks**: 
1. [*Understanding the problem & EDA*](https://www.kaggle.com/vivmankar/understanding-the-problem-eda) 
2. [*ML RandomForestRegressor*](https://www.kaggle.com/vivmankar/ml-randomforestregressor)

## Overview of the Notebook

In this notebook we will discuss the transfer learning approch to the problem 

#### Data preprocessing

>   1. Create the dataset( tf.data.dataset )
>   2. Batching ( To speedup the treaning ) 
>   2. Standardize the data ( To speedup the treaning )
>   3. Configure the dataset for performance ( To speedup the treaning )
>   4. Data augmentation( adding an augmentation to the model ) 

#### Model Building 

>   1. Load the base model ( ResNet152V2 ) 
>   2. Develope a custom model class ( ImageModel(tf.keras.Model) ) 
>   3. Update last layer activation to ReLU(max_value = 100 ) // this helps improving performence 
>   4. Compile model and add callbacks ( Save-Checkpoint, Early Stopping ) 
>   5. Train Model ( MAE on validation split : 15.1742 ) 


### Imports

In [ ]:
import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt 
import seaborn as sns 

import os 
import cv2
import random
 
import tensorflow as tf
from sklearn.model_selection import train_test_split 

import warnings 
warnings.filterwarnings("ignore")

In [ ]:
print(tf.__version__)

## Data 

In [ ]:
os.listdir('../input/petfinder-pawpularity-score')

In [ ]:
data = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
ss = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')

In [ ]:
data_dir = "../input/petfinder-pawpularity-score/train/"
test_dir = "../input/petfinder-pawpularity-score/test/"

In [ ]:
data.shape

In [ ]:
data.head()

### Visualize the data

In [ ]:
_, axs = plt.subplots( 2, 2, figsize=(15, 12))

axs = axs.flatten()
col = data.columns.tolist() 

for a, ax in zip(data.sample(4).iterrows(), axs):
    img = cv2.imread(data_dir + f'{a[1][0]}.jpg')
    img = cv2.resize(img, (600, 600))
    other_info = [col[i] for i in range(13) if a[1][i] == 1 ]
    ax.grid(False)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(img)
    ax.set_title(f'Id: {a[0]}, Pawpularity : {a[1][13]}, ' + ", ".join(other_info), fontsize= 12, fontweight='bold' )
    
plt.show()

## Data preprocessing

 

In [ ]:
data = data[["Id","Pawpularity"] ]  

In [ ]:
train,val  = train_test_split( data, test_size=0.2) ## Approx 500 images for validation 

In [ ]:
train.shape

In [ ]:
val.shape

In [ ]:
filenames = tf.constant(train.Id.map(lambda x : data_dir + f'{x}.jpg' ).tolist())
labels = tf.constant( train.Pawpularity.tolist())
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

In [ ]:
val_filenames = tf.constant(val.Id.map(lambda x : data_dir + f'{x}.jpg' ).tolist())
val_labels = tf.constant( val.Pawpularity.tolist() )
val_dataset = tf.data.Dataset.from_tensor_slices((val_filenames, val_labels))

In [ ]:
list(dataset.as_numpy_iterator())[:5]

### File names to images

In [ ]:
### Hyperparams 

BATCH_SIZE = 64
IMG_SIZE = (224, 224) 


In [ ]:
def _parse_function(filename, output ):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string)
    image_resized = tf.image.resize(image_decoded, IMG_SIZE)
    return image_resized, output

In [ ]:
dataset = dataset.map(_parse_function)
val_dataset = val_dataset.map(_parse_function)

### Batching 

In [ ]:
dataset = dataset.batch(BATCH_SIZE) 
val_dataset = val_dataset.batch(BATCH_SIZE) 

### Standardize the data

The RGB channel values are in the [0, 255] range. This is not ideal for a neural network; in general we should seek to make your input values small.

Here, we will standardize values to be in the [0, 1] range by using tf.keras.layers.Rescaling

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
dataset = dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))

### Configure the dataset for performance
* **Dataset.cache** keeps the images in memory after they're loaded off disk during the first epoch. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache.
* **Dataset.prefetch** overlaps data preprocessing and model execution while training.
 

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
dataset = dataset.cache().shuffle(500).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE) ## We dont need to shuffel the validation data 

#### Lets check a datapoint 

In [ ]:
# # Print one key val pair 

batch_data = list(next(dataset.as_numpy_iterator()))
print(batch_data[0][0])
print(batch_data[1][0])

### Data Augmentation 

> This helps expose the model to more aspects of the data and generalize better.

In [ ]:
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal",
                      input_shape=(IMG_SIZE[0],
                                  IMG_SIZE[1],
                                  3)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

#### Let's visualize what a few augmented examples look like by applying data augmentation to the same image several times

In [ ]:
plt.figure(figsize=(10, 10))

for images, _ in dataset.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        # we are multiplaying the numpy array to 255 because we had normalized the dataset to [ 0, 1] 
        plt.imshow((augmented_images[0].numpy()*255).astype("uint8")) 
        plt.axis("off")

## Model Developement 

We will use ResNet152V2 as a base model, other opctions for pretrained models can be found [here](https://keras.io/api/applications/)
 

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.ResNet152V2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
#Freeze the convolutional base
base_model.trainable = False

In [ ]:
# base_model.summary()

In [ ]:
class ImageModel(tf.keras.Model):

    def __init__(self):

        super(ImageModel, self).__init__()

        self.input_l = tf.keras.layers.InputLayer( input_shape = IMG_SIZE + (3,)  ) 
        
        self.base_model = base_model
        self.preprocess_input = tf.keras.applications.resnet_v2.preprocess_input 
        self.data_augmentation = tf.keras.Sequential([
                                tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
                                tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
                                ])
        self.gap = tf.keras.layers.GlobalAveragePooling2D() ##  ( batch_size , 2048 )

        self.activation = tf.keras.layers.LeakyReLU( alpha=0.15 )
        self.activation_final = tf.keras.layers.ReLU(max_value = 100 ) # since the maximum score can be 100 
        
        self.dense_1 = tf.keras.layers.Dense(512, activation= self.activation )
        self.dense_2 = tf.keras.layers.Dense(128, activation= self.activation )
        self.dense_3 = tf.keras.layers.Dense(32, activation= self.activation  )
        self.final = tf.keras.layers.Dense(1, activation= self.activation_final )

        
    def call(self, input_tensor):

        x = self.input_l(input_tensor)
        x = self.data_augmentation(x)
        x = self.preprocess_input(x)
        x = self.base_model(x)
        x = self.gap(x)

        x = self.dense_1(x)
        x = self.dense_2(x)
        x = self.dense_3(x)
        x = self.final(x)
 
        return  x

In [ ]:
def create_model():
    
    model = ImageModel()
    
    model.compile(
        optimizer='adam', 
        loss="mse", # Mean squared error 
        metrics=["mae"] # Mean Absolute Error
      )
    
    return model 

### Compile and train the model

In [ ]:
CNN_regressor = create_model()

In [ ]:
epochs = 20

checkpoint_path = "cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

es_callback = tf.keras.callbacks.EarlyStopping(
                                monitor='val_mae',
                                patience=3,
                                verbose=1,
                                restore_best_weights=True)

history = CNN_regressor.fit(
                    dataset,
                    validation_data = val_dataset, 
                    epochs=epochs,
                    callbacks = [cp_callback , es_callback ] ,
                    )

### Load the saved model 

In [ ]:
saved_checkpoint_path = "cp.ckpt"

In [ ]:
# Create a basic model instance
model = create_model()

# Loads the weights
model.load_weights(saved_checkpoint_path)

## Predict on test data

In [ ]:
def _parse_function_test(filename ):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string)
    image_resized = tf.image.resize(image_decoded, IMG_SIZE)
    return image_resized 

In [ ]:
test_filenames = tf.constant(test.Id.map(lambda x : test_dir + f'{x}.jpg' ).tolist())
test_dataset = tf.data.Dataset.from_tensor_slices((test_filenames))
test_dataset = test_dataset.map(_parse_function_test)
test_dataset = test_dataset.map(lambda x: (normalization_layer(x)))

In [ ]:
test_dataset = test_dataset.batch(len(test))

In [ ]:
predictions = model.predict(test_dataset)

In [ ]:
ss.head()

In [ ]:
submission = pd.DataFrame()
submission["Id"] = test["Id"]
submission["Pawpularity"]= predictions

In [ ]:
submission.head()

In [ ]:
ss.columns.equals(submission.columns)

In [ ]:
submission.to_csv('submission.csv', index=False)